# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD model for fitting the actual Chilean data in order to make projections in VMI and bed usage, amount of infected and deaths. This model uses the SEIRHVD 4.1 version. A visual representation of the model is available at: [SEIRHVD Miró Model](https://miro.com/app/board/o9J_ktzN4xA=/)

This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from datetime import datetime
from numpy import linalg as LA
import multiprocessing
from joblib import Parallel, delayed
from datetime import timedelta

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD5 import SEIRHVD 
from Quarantine import Quarantine
from Quarantine import SeroPrevDynamics
from importdata import ImportData
import Events

import json
from pandas.core.base import DataError
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import matplotlib

from datetime import date

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [3]:
# First simulation Initial date
initdate = datetime(2020,12,3)
# Date for change in SeroPrevalence
SPchange_date = datetime(2020,7,25)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days

fiestaspatrias = datetime(2020,9,18)
fiestaspatrias_day = (fiestaspatrias-initdate).days
plebiscito = datetime(2020,10,25)
plebiscito_day = (plebiscito-initdate).days

navidad = datetime(2020,12,25)
navidad_day = (navidad-initdate).days

newyear = datetime(2020,12,31)
newyear_day = (newyear-initdate).days

today = datetime.today()
today_day = (today-initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. state region is represented by cut = 14.

In [4]:
def dataretriever(user=None,password=None):
    # Esta funcion devolvera una funcion de request        
    def request(endpoint):
        if user:
            aux = requests.get('https://api.cv19gm.org/'+endpoint, auth=HTTPBasicAuth(user, password))
            if aux.status_code == 401:
                raise Exception('Wrong Credentials')
            else:
                print('Logged in successfully')
        else:
            aux = requests.get('http://192.168.2.223:5006/'+endpoint)
        return aux
    return request
def timeJStoPy(t):
    return datetime.strptime(t[:10],'%Y-%m-%d')
def timetxttoDatetime(t):
    return datetime.strptime(t,'%Y-%m-%d')

In [5]:
request = dataretriever()

In [6]:
print('Importing data from all regions')
tstate = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16']
#tstate = '13101'
# Import Data
state = ImportData(tstate=tstate,initdate = initdate)
state.importdata()

state.importDailyInfectedNacional()
state.name = 'Chile'

Importing data from all regions
Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing ICU Beds Data
Importing Deaths by DEIS
Importing Active Infected by Minciencia
Done
Importing Daily Infected


### Datos de Hospitalizados a fitear

In [7]:
state.Hr = 1000
state.Hr_tot = [10000,10000]
state.Vr = state.UCI_use_covid
state.Vr_tot = state.UCI_capacity
state.UCI_non_covid = state.UCI_use_noncovid
state.sochimi_tr = state.UCI_tr#[-2:]
state.sochimi_dates = state.UCI_dates

## Infectados Diarios

In [8]:
endpoint = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales.csv'
casosnuevos = pd.read_csv(endpoint)
I_d = casosnuevos[casosnuevos['Fecha']=='Casos nuevos totales'].transpose()
dates_I_d = [timetxttoDatetime(I_d.axes[0][i]) for i in range(1,len(I_d.axes[0]))]
I_d = np.array(I_d.iloc[1:][6])
I_ac = np.cumsum(I_d)

index_ac = np.where(np.array(dates_I_d) >= initdate)[0][0]
I_ac = I_ac[index_ac:]
I_ac_dates = dates_I_d[index_ac:]
I_ac_tr = [(i-initdate).days for i in I_ac_dates]

index = np.where(np.array(dates_I_d) >= initdate-timedelta(days=7))[0][0]
dates_I_d = dates_I_d[index:]
I_d = I_d[index:]
weekdays = [date.weekday(dates_I_d[i]) for i in range(len(dates_I_d))]
I = pd.DataFrame({'dates':dates_I_d,'I':I_d,'weekdays':weekdays})
I_week = I[(I['weekdays']>2) | (I['weekdays']==0)]
I_week_ra = I_week['I'].rolling(5).mean()[I_week['dates']>=initdate]
I_week_ra_dates = I_week['dates'][I_week['dates']>=initdate]
I_week_ra_tr = [(i-initdate).days for i in I_week_ra_dates]
index_date = np.where(np.array(dates_I_d) >= initdate)[0][0]

In [9]:
initdate

datetime.datetime(2020, 12, 3, 0, 0)

# Parameters

In [10]:
# Total simulation time
tsim = 1000
#t_sp = (SPchange_date - initdate).days

beta = 0.023
mu = 0.298
k_I = 0
k_R = 0#10
SeroPrevFactor = 0.2#1
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Subreport
Fraction of Infected detected/reported  

$Imi_{det}$: Fraction of Mild detected   
$Ias_{det}$: fraction of asymptomatic detected  

If both are 1, means that all infected are detected so 
\begin{align}
I_{det} = I
\end{align}

In [11]:
Imi_det = 0.4
Ias_det = 0.1
Ise_det = 1
Icr_det = 1

# Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [12]:
mitdadvacaciones = datetime(2021,2,15)
mitadvacaciones_day = (mitdadvacaciones - initdate).days
cuarentena = datetime(2021,4,11)
cuarentena_day = (cuarentena - initdate).days

# Quarantines 
first_mob = 0.45
second_mob = 0.57
third_mob = 0.59 #0.612
fourth_mob = 0.59 #0.612

days=[[0,newyear_day+3],[newyear_day+3,mitadvacaciones_day],[mitadvacaciones_day,cuarentena_day],[cuarentena_day,tsim]]
alpha_new = Events.Events(values=[first_mob,second_mob,third_mob,fourth_mob],days=days)

In [13]:
# Plot Quarantines
if False:
    t = np.array(np.arange(0,240,0.1))
    alphaplot= [[j(i) for i in t] for j in [alpha_new]]
    for f in alphaplot:
        plt.plot(t,f)
    #for f in alphaplot_UK:
        #plt.plot(t,f,linestyle='dashed')    
    plt.tick_params(labelsize=18)
    plt.legend(loc=0)
    plt.title('Mobility-beta',fontsize=20)
    plt.show()


## SeroPrevalence Dynamics
The susceptibles increase through a step function which last for the specified amount of days. Throughout these days it adds a "dailyincrease" amount of persons each day.

The total increase of people is increasedays*dailyincrease

In [14]:
def chisum(a,b):
    def aux(t):
        return a(t)+b(t)
    return aux

In [15]:
chi = []
increasedays = 62#(fiestaspatrias - SPchange_date).days + 0
renewalFactor = 0.6
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = (plebiscito-initdate).days -3
chi0 = SeroPrevDynamics(t_sp_temp,t_sp_temp+increasedays*0.8,t_sp_temp+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)
#chi = [chi0]
#renewalFactor = [renewalFactor]
chi.append(chi0)


# Navidad
increasedays = 2
renewalFactor = 0.65
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays

chi1 = SeroPrevDynamics(navidad_day,navidad_day+increasedays*0.8,navidad_day+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)

#renewalFactor = [renewalFactor]
#chi = Events.functionAddition([chi0,chi1])
chi.append(chi1)


# Año nuevo
delay = 0
increasedays = 7
renewalFactor_ny = 0.25
dailyincrease = state.population*SeroPrevFactor*renewalFactor_ny/increasedays

chi2 = SeroPrevDynamics(newyear_day+delay,newyear_day+delay+increasedays*0.05,newyear_day+delay+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)

#renewalFactor = [renewalFactor]
chi.append(chi2)

chi = Events.functionAddition(chi)

#### Contraction during holidays
delay = 0
hd_contraction = datetime(2021,1,19) #+ timedelta(days=30)
hd_contraction_day = (hd_contraction-initdate).days
# Increase date 
#increasedate = today
#increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 10
decrease = -0.55
# Daily amount of people
dailyincrease = state.population*SeroPrevFactor*decrease/increasedays

chi4 = SeroPrevDynamics(hd_contraction_day,hd_contraction_day+delay+increasedays*0.0001,hd_contraction_day+delay+increasedays,dailyincrease)       
chi = [chisum(chi,chi4)]


#### Second Expansion during holidays

delay = 0
hd_expansion = datetime(2021,2,13) #+ timedelta(days=30)
hd_expansion_day = (hd_expansion-initdate).days
# Increase date 
#increasedate = today
#increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 33
increase = 1.4
# Daily amount of people
dailyincrease = state.population*SeroPrevFactor*increase/increasedays

chi5 = SeroPrevDynamics(hd_expansion_day,hd_expansion_day+delay+increasedays*0.0001,hd_expansion_day+delay+increasedays,dailyincrease)       
chi = [chisum(chi[0],chi5)]


#### Contraction due to lockdown
delay = 0
hd_expansion = datetime(2021,4,11) #+ timedelta(days=30)
hd_expansion_day = (hd_expansion-initdate).days
# Increase date 
#increasedate = today
#increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 15
increase = -0.5
# Daily amount of people
dailyincrease = state.population*SeroPrevFactor*increase/increasedays

chi6 = SeroPrevDynamics(hd_expansion_day,hd_expansion_day+delay+increasedays*0.0001,hd_expansion_day+delay+increasedays,dailyincrease)       
chi = [chisum(chi[0],chi6)]

#### Plot Susceptible Dynamics

In [16]:
if True:
    chiplot = [chi[0](t) for t in np.arange(0,tsim,0.1)]
    plt.title('Tasa Ingreso de Susceptibles',fontsize=20)
    
    plt.plot(np.arange(0,tsim,0.1),chiplot)#,label='RenewalFactor: '+str(renewalFactor[i]))
    plt.axvline(x = navidad_day, linestyle = 'dotted',color = 'red',label='Navidad')
    plt.axvline(x = newyear_day, linestyle = 'dotted',color = 'blue',label='Año Nuevp')
    plt.axvline(x = today_day, linestyle = 'dotted',color = 'green',label='hoy')

    plt.xlim(0,150)
    plt.tick_params(labelsize=18)
    plt.legend(loc=0,fontsize=15)
    plt.show()

# State transition parameters:

In [17]:
#state parameters: 
pE_Ias = 0.43 # Transition from exposed to Asymptomatic Infected
tE_Ias = 3.0
pE_Imi = 0.53 # Transition from exposed to  Mild Infected
tE_Imi = 3.0
pE_Icr = 0.0065 # Transition from exposed to  Critical Infected
tE_Icr = 3.0
pE_Ise = 0.0335# Transition from exposed to  Serious Infected
tE_Ise = 3.0
pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered
tIas_R = 5.0
pImi_R = 1.0  # Transition from Mild Infected to Recovered
tImi_R = 5.0
pIse_Hse = 1 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)
tIse_Hse = 5.0
pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)
tIse_D = 3.0
pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)
tIcr_V = 3.0
pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)
tIcr_D = 3.0
pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered
tHse_R = 3.5
pHse_V = 0.03# Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)
tHse_V = 4.0
pHse_D = 0.85# Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)
tHse_D = 4.0
pV_Hout = 0.45  # Transition from Ventilators to Hospital Recovery (Hout)
tV_Hout = 16.0
pV_D = 0.55 # Transition from Ventilators to Death
tV_D = 12.0
pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered
tHout_R = 10.0

# Simulation

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

#alpha[0]
simulation = [SEIRHVD(tsim,beta,mu,alpha_new,k_I=k_I,k_R = k_R, chi = chi[i],
                      Htot=10000,Vtot=int(state.UCI_capacity[0]),H0=1000,V0=state.UCI_use_covid[0],B0=state.Br[0],D0=state.Dr[0]
                      ,R0=state.I_ac_r[0]-state.I_ac_r[0]*0.1,I0=state.Ir[0],I_d0=state.I_d_r[0],I_ac0=state.I_ac_r[0],
                      SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,initdate = initdate,
                      Imi_det = Imi_det,Ias_det = Ias_det,population = state.population) for i in range(len(chi))]

In [18]:
simulation = [SEIRHVD(tsim,beta,mu,alpha_new,k_I=k_I,k_R = k_R, chi = chi[i],                      
                      SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,initdate = initdate,
                      Imi_det = Imi_det,Ias_det = Ias_det,RealIC=state) for i in range(len(chi))]

InitialCondition Object Data


Modify simulation state parameters

In [19]:
for i in range(len(simulation)):
    simulation[i].pE_Ias=pE_Ias
    simulation[i].tE_Ias=tE_Ias
    simulation[i].pE_Imi=pE_Imi
    simulation[i].tE_Imi=tE_Imi
    simulation[i].pE_Icr=pE_Icr
    simulation[i].tE_Icr=tE_Icr
    simulation[i].pE_Ise=pE_Ise
    simulation[i].tE_Ise=tE_Ise
    simulation[i].pIas_R=pIas_R
    simulation[i].tIas_R =tIas_R
    simulation[i].pImi_R=pImi_R
    simulation[i].tImi_R =tImi_R
    simulation[i].pIse_Hse=pIse_Hse
    simulation[i].tIse_Hse=tIse_Hse
    simulation[i].pIse_D=pIse_D
    simulation[i].tIse_D=tIse_D
    simulation[i].pIcr_V=pIcr_V
    simulation[i].tIcr_V=tIcr_V
    simulation[i].pIcr_D=pIcr_D
    simulation[i].tIcr_D=tIcr_D
    simulation[i].pHse_R=pHse_R
    simulation[i].tHse_R =tHse_R
    simulation[i].pHse_V=pHse_V
    simulation[i].tHse_V=tHse_V
    simulation[i].pHse_D=pHse_D
    simulation[i].tHse_D=tHse_D
    simulation[i].pV_Hout=pV_Hout
    simulation[i].tV_Hout =tV_Hout
    simulation[i].pV_D=pV_D
    simulation[i].tV_D =tV_D
    simulation[i].pHout_R=pHout_R
    simulation[i].tHout_R=tHout_R
    simulation[i].setnewparams()

Compartimental model State parameters changed


### Setting new params

## Run simulation

* **integr_sci**: Fast solver, sometimes stiffness beat it  
* **integr:** Slow solver, more robust with stiffness  

In [20]:
def simulate(simulation,i,tsim):
    simulation[i].integr_sci(0,tsim,0.1)
    return simulation[i]

In [21]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(simulation,i,tsim) for i in range(len(simulation)))
simulation = sims
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=12)]: Done   1 out of   1 | elapsed:    1.8s finished
ready


# Simulation Informe Diario

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

## Parameters

In [22]:
# Total simulation time
beta_InfDiario = 0.022
mu_InfDiario = 0.31#298
k_I = 0
k_R = 0#10
SeroPrevFactor = 0.2#1
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

# Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [23]:
mitdadvacaciones = datetime(2021,2,24)
mitadvacaciones_day = (mitdadvacaciones - initdate).days
cuarentena = datetime(2021,4,11)
cuarentena_day = (cuarentena - initdate).days

# Quarantines 
# inicio
first_mob = 0.46#0.45
# Año nuevo
second_mob = 0.57
# Expansion mitad febrero
third_mob = 0.6#612 #0.612
# Cuarentena
fourth_mob = third_mob #0.612

days=[[0,newyear_day+3],[newyear_day+3,mitadvacaciones_day],[mitadvacaciones_day,cuarentena_day],[cuarentena_day,tsim]]
alpha_InfDiario = Events.Events(values=[first_mob,second_mob,third_mob,fourth_mob],days=days)

In [24]:
# Plot Quarantines
if False:
    t = np.array(np.arange(0,240,0.1))
    alphaplot= [[j(i) for i in t] for j in [alpha_InfDiario]]
    for f in alphaplot:
        plt.plot(t,f)
    #for f in alphaplot_UK:
        #plt.plot(t,f,linestyle='dashed')    
    plt.tick_params(labelsize=18)
    plt.legend(loc=0)
    plt.title('Mobility-beta',fontsize=20)
    plt.show()


## SeroPrevalence Dynamics
The susceptibles increase through a step function which last for the specified amount of days. Throughout these days it adds a "dailyincrease" amount of persons each day.

The total increase of people is increasedays*dailyincrease

In [25]:
def chisum(a,b):
    def aux(t):
        return a(t)+b(t)
    return aux

In [26]:
chi = []
increasedays = 62#(fiestaspatrias - SPchange_date).days + 0
renewalFactor = 1.0 #0.7
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = (plebiscito-initdate).days -3
chi0 = SeroPrevDynamics(t_sp_temp,t_sp_temp+increasedays*0.8,t_sp_temp+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)
#chi = [chi0]
#renewalFactor = [renewalFactor]
chi.append(chi0)


# Navidad
delay = 3
increasedays = 2
renewalFactor = 0.65 #0.65
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays

chi1 = SeroPrevDynamics(navidad_day+delay,navidad_day+delay+increasedays*0.8,navidad_day+delay+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)

#renewalFactor = [renewalFactor]
#chi = Events.functionAddition([chi0,chi1])
chi.append(chi1)


# Año nuevo
delay = 3
increasedays = 7
renewalFactor_ny = 0.25
dailyincrease = state.population*SeroPrevFactor*renewalFactor_ny/increasedays

chi2 = SeroPrevDynamics(newyear_day+delay,newyear_day+delay+increasedays*0.05,newyear_day+delay+increasedays,dailyincrease,form='line')
#chi0 = SeroPrevDynamics(t_sp,t_sp+increasedays*1,t_sp+increasedays,dailyincrease,'quadratic',df = 1)

#renewalFactor = [renewalFactor]
chi.append(chi2)

chi = Events.functionAddition(chi)

#### Contraction during holidays
delay = 5
hd_contraction = datetime(2021,1,19) #+ timedelta(days=30)
hd_contraction_day = (hd_contraction-initdate).days
# Increase date 
#increasedate = today
#increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 10
decrease = -0.50
# Daily amount of people
dailyincrease = state.population*SeroPrevFactor*decrease/increasedays

chi4 = SeroPrevDynamics(hd_contraction_day+delay,hd_contraction_day+delay+increasedays*0.0001,hd_contraction_day+delay+increasedays,dailyincrease)       
chi = [chisum(chi,chi4)]


#### Second Expansion during holidays

delay = 7
hd_expansion = datetime(2021,2,13) #+ timedelta(days=30)
hd_expansion_day = (hd_expansion-initdate).days
# Increase date 
#increasedate = today
#increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 7
increase = 1#1#0.7#0.7#1.4
# Daily amount of people
dailyincrease = state.population*SeroPrevFactor*increase/increasedays

chi5 = SeroPrevDynamics(hd_expansion_day+delay,hd_expansion_day+delay+increasedays*0.0001,hd_expansion_day+delay+increasedays,dailyincrease)       
chi = [chisum(chi[0],chi5)]


#### Contraction due to lockdown
delay = 0
hd_expansion = datetime(2021,4,11) #+ timedelta(days=30)
hd_expansion_day = (hd_expansion-initdate).days
# Increase date 
#increasedate = today
#increaseday = (increasedate - initdate).days
# Days for susceptible increase
increasedays = 15
increase = 0#-0.5
# Daily amount of people
dailyincrease = state.population*SeroPrevFactor*increase/increasedays

chi6 = SeroPrevDynamics(hd_expansion_day,hd_expansion_day+delay+increasedays*0.0001,hd_expansion_day+delay+increasedays,dailyincrease)       
chi = [chisum(chi[0],chi6)]

#### Plot Susceptible Dynamics

In [27]:
if False:
    chiplot = [chi[0](t) for t in np.arange(0,tsim,0.1)]
    plt.title('Tasa Ingreso de Susceptibles',fontsize=20)
    
    plt.plot(np.arange(0,tsim,0.1),chiplot)#,label='RenewalFactor: '+str(renewalFactor[i]))
    plt.axvline(x = navidad_day, linestyle = 'dotted',color = 'red',label='Navidad')
    plt.axvline(x = newyear_day, linestyle = 'dotted',color = 'blue',label='Año Nuevp')
    plt.axvline(x = today_day, linestyle = 'dotted',color = 'green',label='hoy')

    plt.xlim(0,150)
    plt.tick_params(labelsize=18)
    plt.legend(loc=0,fontsize=15)
    plt.show()

### Simulation constructor

In [28]:
import copy

In [29]:
state_InfDiario = copy.deepcopy(state)
state_InfDiario.I_d_r = list(I_week_ra)
state_InfDiario.I_d_r_dates = list(I_week_ra_dates)
state_InfDiario.I_d_r_tr = I_week_ra_tr

state_InfDiario.I_ac_r = I_ac
state_InfDiario.I_ac_r_dates = I_ac_dates
state_InfDiario.I_ac_r_tr = I_ac_tr


In [30]:
simulation_InfDiario = [SEIRHVD(tsim,beta_InfDiario,mu_InfDiario,alpha_InfDiario,k_I=k_I,k_R = k_R, chi = chi[i],                      
                      SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,initdate = initdate,
                      Imi_det = Imi_det,Ias_det = Ias_det,RealIC=state_InfDiario) for i in range(len(chi))]

InitialCondition Object Data


Modify simulation state parameters

In [31]:
for i in range(len(simulation_InfDiario)):
    simulation_InfDiario[i].pE_Ias=pE_Ias
    simulation_InfDiario[i].tE_Ias=tE_Ias
    simulation_InfDiario[i].pE_Imi=pE_Imi
    simulation_InfDiario[i].tE_Imi=tE_Imi
    simulation_InfDiario[i].pE_Icr=pE_Icr
    simulation_InfDiario[i].tE_Icr=tE_Icr
    simulation_InfDiario[i].pE_Ise=pE_Ise
    simulation_InfDiario[i].tE_Ise=tE_Ise
    simulation_InfDiario[i].pIas_R=pIas_R
    simulation_InfDiario[i].tIas_R =tIas_R
    simulation_InfDiario[i].pImi_R=pImi_R
    simulation_InfDiario[i].tImi_R =tImi_R
    simulation_InfDiario[i].pIse_Hse=pIse_Hse
    simulation_InfDiario[i].tIse_Hse=tIse_Hse
    simulation_InfDiario[i].pIse_D=pIse_D
    simulation_InfDiario[i].tIse_D=tIse_D
    simulation_InfDiario[i].pIcr_V=pIcr_V
    simulation_InfDiario[i].tIcr_V=tIcr_V
    simulation_InfDiario[i].pIcr_D=pIcr_D
    simulation_InfDiario[i].tIcr_D=tIcr_D
    simulation_InfDiario[i].pHse_R=pHse_R
    simulation_InfDiario[i].tHse_R =tHse_R
    simulation_InfDiario[i].pHse_V=pHse_V
    simulation_InfDiario[i].tHse_V=tHse_V
    simulation_InfDiario[i].pHse_D=pHse_D
    simulation_InfDiario[i].tHse_D=tHse_D
    simulation_InfDiario[i].pV_Hout=pV_Hout
    simulation_InfDiario[i].tV_Hout =tV_Hout
    simulation_InfDiario[i].pV_D=pV_D
    simulation_InfDiario[i].tV_D =tV_D
    simulation_InfDiario[i].pHout_R=pHout_R
    simulation_InfDiario[i].tHout_R=tHout_R
    simulation_InfDiario[i].setnewparams()

Compartimental model State parameters changed


### Setting new params

## Run simulation

* **integr_sci**: Fast solver, sometimes stiffness beat it  
* **integr:** Slow solver, more robust with stiffness  

In [32]:
def simulate(simulation,i,tsim):
    simulation[i].integr_sci(0,tsim,0.1)
    return simulation[i]

In [33]:
num_cores = multiprocessing.cpu_count()
sims_InfDiario = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(simulation_InfDiario,i,tsim) for i in range(len(simulation_InfDiario)))
simulation_InfDiario = sims_InfDiario
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.5s
[Parallel(n_jobs=12)]: Done   1 out of   1 | elapsed:    1.5s finished
ready


# Analysis

In [34]:
saveplot = False
if saveplot:
    %matplotlib inline
#%matplotlib inline
%matplotlib tk
# Big Screen
plt.rcParams["figure.figsize"] = 38.40, 20.56 #19.20,10.80#8,6
# Small Screen
#plt.rcParams["figure.figsize"] = 19.2, 10.80 #19.20,10.80#8,6
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif":["Arial"]})


## Grid Plot

In [35]:
# Days to plot
days = 500
t_end = 50

enddate = initdate + timedelta(days=365)
enddate = datetime(2021,9,30)

In [36]:
dates = [[initdate+timedelta(days=int(i)) for i in simulation[j].t] for j in range(len(simulation))]

In [37]:
SeroPrevalence = [simulation[i].I_ac[-1]/simulation[i].population for i in range(len(simulation))]
SeroPrevalenceDet = [simulation[i].I_ac_det[-1]/simulation[i].population for i in range(len(simulation))]

In [38]:
pĺotlinewidth = 4
datelinewidth = 3

### Pantalla

In [39]:
#4k
fontsize_title = 22
fontsize_suptitle = 25
fontsize_label = 18
fontsize_legend = 14

In [40]:
#1080
fontsize_title = 15
fontsize_suptitle = 16
fontsize_label = 12
fontsize_legend = 8

pĺotlinewidth = 2
datelinewidth = 1

In [41]:
renewalFactor = [0]

In [42]:
sim_InfDiario = True
sim_InfEPI = True

In [43]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(renewalFactor)))

fig, axs = plt.subplots(2, 2)

for i in range(len(simulation)):
    if sim_InfEPI:
        axs[0,0].plot(dates[i],simulation[i].I_ac_det,linewidth=pĺotlinewidth,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    if sim_InfDiario:
        axs[0,0].plot(dates[i],simulation_InfDiario[i].I_ac_det,color=colors[i],linestyle='dashed',linewidth=pĺotlinewidth)
axs[0,0].set_title('Infectados acumulados',fontsize=fontsize_title)
axs[0,0].set_xlim(initdate,enddate)
#axs[0,0].set_ylim(0,1.5e6)



axs[0,0].axvline(x = datetime(2020,12,25), linestyle = 'dotted',color = 'red',label='Navidad',linewidth=datelinewidth)
axs[0,0].axvline(x = datetime(2020,12,31), linestyle = 'dotted',color = 'blue',label='Año Nuevo',linewidth=datelinewidth)
axs[0,0].axvline(x = datetime(2021,2,15), linestyle = 'dotted',color = 'green',label='15 Febrero',linewidth=datelinewidth)


#axs[0,0].plot([], [], ' ', label='RF: RenewalFactor')
#axs[0,0].plot([], [], ' ', label='SP_R: SeroPrevalence Real')
#axs[0,0].plot([], [], ' ', label='SP_D: SeroPrevalence Detected')

if sim_InfEPI:
    axs[0,0].scatter(state.I_ac_r_dates,state.I_ac_r,color='black',label='Informe EPI',zorder=10)

if sim_InfDiario:
    # cambiar
    axs[0,0].scatter(I_ac_dates,I_ac,color='grey',label='Informe Diario',zorder=10)

axs[0,0].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[0,0].legend(loc=0)

axs[0,0].grid(color='grey', linestyle='dotted', linewidth=0.5)


# Accumulated deaths
for i in range(len(simulation)):
    if sim_InfEPI:
        axs[0,1].plot(dates[i],simulation[i].B,label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i],linewidth=pĺotlinewidth)
    if sim_InfDiario:
        axs[0,1].plot(dates[i],simulation_InfDiario[i].B,color=colors[i],linestyle='dashed',linewidth=pĺotlinewidth)
    
axs[0,1].set_title('Muertes Acumuladas',fontsize=fontsize_title)
axs[0,1].set_xlim(initdate,enddate)
#axs[0,1].set_ylim(0,1e4)

axs[0,1].axvline(x = datetime(2020,12,25), linestyle = 'dotted',color = 'red',label='Navidad',linewidth=datelinewidth)
axs[0,1].axvline(x = datetime(2020,12,31), linestyle = 'dotted',color = 'blue',label='Año Nuevo',linewidth=datelinewidth)
axs[0,1].axvline(x = datetime(2021,2,15), linestyle = 'dotted',color = 'green',label='15 Febrero',linewidth=datelinewidth)

axs[0,1].scatter(state.Br_dates,state.Br,label='Datos',color='black',zorder=10)

axs[0,1].fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs[0,1].legend(loc=0)


for i in range(len(simulation)):
    if sim_InfEPI:
        axs[1,0].plot(dates[i],simulation[i].I_d_det,label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i],linewidth=pĺotlinewidth)
    if sim_InfDiario:
        axs[1,0].plot(dates[i],simulation_InfDiario[i].I_d_det,color=colors[i],linestyle='dashed',linewidth=pĺotlinewidth)    
axs[1,0].set_title('Nuevos Infectados Diarios',fontsize=fontsize_title)
axs[1,0].set_xlim(initdate,enddate)
#axs[1,0].set_ylim(0,1e4)

if sim_InfEPI:
    axs[1,0].scatter(state.I_d_r_dates,state.I_d_r,label='Informe EPI',color='black',zorder=5)
if sim_InfDiario:
    axs[1,0].scatter(I_week_ra_dates,I_week_ra,label='Informe diario (RA5D)',color='red',zorder=5)  
    axs[1,0].plot(dates_I_d,I_d,label='Informe diario',color='blue',zorder=10,linestyle='dashed')
    
#axs[1,0].axvline(x = SPchange_date, linestyle = 'dotted',color = 'grey',label='Inicio Apertura')
#axs[1,0].axvline(x = datetime(2020,8,22), linestyle = 'dotted',color = 'grey',label='Inicio Cuarentena')
#axs[1,0].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
#axs[1,0].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito',linewidth=datelinewidth)
axs[1,0].axvline(x = datetime(2020,12,25), linestyle = 'dotted',color = 'red',label='Navidad',linewidth=datelinewidth)
axs[1,0].axvline(x = datetime(2020,12,31), linestyle = 'dotted',color = 'blue',label='Año Nuevo',linewidth=datelinewidth)
axs[1,0].axvline(x = datetime(2021,2,15), linestyle = 'dotted',color = 'green',label='15 Febrero',linewidth=datelinewidth)

#axs[1,0].axvline(x = datetime(2020,7,18), linestyle = 'dotted',color = 'grey',label='Cambio a fase 3')
#axs[1,0].axvline(x = datetime(2020,9,11), linestyle = 'dashed',color = 'grey',label='Cambio a fase 4')
#axs[1,0].axvline(x = datetime(2020,10,3), linestyle = 'dashdot',color = 'grey',label='Cambio a fase 2')


axs[1,0].legend(loc=0)


for i in range(len(simulation)):
    if sim_InfEPI:
        axs[1,1].plot(dates[i],simulation[i].V,label='RenewalFactor: '+str(renewalFactor[i]),color=colors[i],linewidth=pĺotlinewidth)
    #axs[1,1].plot(dates[i],simulation[i].V_need,linestyle='dashed',color=colors[i])
    if sim_InfDiario:
        axs[1,1].plot(dates[i],simulation_InfDiario[i].V,color=colors[i],linestyle='dashed',linewidth=pĺotlinewidth)    
axs[1,1].set_title('Uso UCI',fontsize=fontsize_title)
axs[1,1].set_xlim(initdate,enddate)
#axs[1,1].set_ylim(0,3e3)

axs[1,1].scatter(state.UCI_dates,state.UCI_use_covid,label='Datos',color='black',zorder=10)
#axs[1,1].plot(state.UCI_dates,state.UCI_capacity,label='UCI Capacity', linestyle = 'dashed', color = 'grey')
axs[1,1].plot(simulation[0].dates,simulation[0].V_cap,label='UCI Capacity', linestyle = 'dashed', color = 'black')


#axs[1,1].axvline(x = fiestaspatrias, linestyle = 'dotted',color = 'red',label='18 Sept')
#axs[1,1].axvline(x = plebiscito, linestyle = 'dotted',color = 'blue',label='Plebiscito',linewidth=datelinewidth)
axs[1,1].axvline(x = datetime(2020,12,25), linestyle = 'dotted',color = 'red',label='Navidad',linewidth=datelinewidth)
axs[1,1].axvline(x = datetime(2020,12,31), linestyle = 'dotted',color = 'blue',label='Año Nuevo',linewidth=datelinewidth)
axs[1,1].axvline(x = datetime(2021,2,15), linestyle = 'dotted',color = 'green',label='15 Febrero',linewidth=datelinewidth)



# Format and shit

# Grid
axs[0,0].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[0,1].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[1,0].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[1,1].grid(color='grey', linestyle='dotted', linewidth=0.5)


# Date format
axs[0,0].fmt_xdata = mdates.DateFormatter('%d-%m-%Y')
axs[0,1].fmt_xdata = mdates.DateFormatter('%d-%m-%Y')
axs[1,0].fmt_xdata = mdates.DateFormatter('%d-%m-%Y')
axs[1,1].fmt_xdata = mdates.DateFormatter('%d-%m-%Y')

# Y axis
#axs[0,0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[0,1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[1,0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[1,1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# Font-size
axs[0,0].tick_params(labelsize=fontsize_label)
axs[0,1].tick_params(labelsize=fontsize_label)
axs[1,0].tick_params(labelsize=fontsize_label)
axs[1,1].tick_params(labelsize=fontsize_label)

# Legend
axs[0,0].legend(loc=0,fontsize=fontsize_legend)
axs[0,1].legend(loc=1,fontsize=fontsize_legend)
axs[1,0].legend(loc=1,fontsize=fontsize_legend)
axs[1,1].legend(loc=1,fontsize=fontsize_legend)


fig.suptitle(state.name,fontsize=fontsize_suptitle)
fig.show()

In [44]:
plt.plot(dates[0],simulation_InfDiario[0].I)
plt.show()

In [45]:
simulation_InfDiario[0].I_ac[0]

1775003.3898305085

In [46]:
simulation_InfDiario[0].prevalence_total[(datetime(2021,3,9)-initdate).days]

0.14792398194423123

In [47]:
print('stop')

stop


In [48]:
print('stop')

stop


In [49]:
plt.scatter(state.I_d_r_dates,np.log(state.I_d_r),label='Informe EPI',color='black',zorder=5)
plt.scatter(I_week_ra_dates,np.log(I_week_ra),label='Informe diario (RA5D)',color='red',zorder=5)  
plt.legend(loc=0)
plt.show() 

# Simulación control

In [50]:
tsim = 10000
mu = 0.1

In [51]:
def alphacontrol(t):
    return 1

In [52]:
simulation_control = SEIRHVD(tsim,0.13,mu,alphacontrol,k_I=k_I,k_R = k_R, chi = 0,                      
                      expinfection=expinfection,initdate = initdate,
                      RealIC=state)

InitialCondition Object Data


In [53]:
beta = [0.01,0.05,0.1,0.15]
sims = []
for i in beta:
    sims.append(SEIRHVD(tsim,i,mu,alphacontrol,Htot=10000,Vtot=10000,
    H0=0,V0=0,B0=0,D0=0,R0=0,I0=100,I_d0=10,I_ac0=100,SeroPrevFactor=1,expinfection=0,
    population=1000000))
    

In [54]:
linestyle = ['dashed','dotted','solid','dashdot']

In [55]:
for i in sims:
    i.integr_sci(0,tsim,0.1)

In [56]:
for i in range(len(sims)):
    #plt.plot(sims[i].t,sims[i].S,label='S beta: '+str(beta[i]),linestyle=linestyle[i])
    #plt.plot(sims[i].t,sims[i].I,label='I beta: '+str(beta[i]),linestyle=linestyle[i])
    plt.plot(sims[i].t,sims[i].R,label='R beta: '+str(beta[i]),linestyle=linestyle[i])
    plt.plot(sims[i].t,sims[i].B,label='B beta: '+str(beta[i]),linestyle=linestyle[i])
    #plt.plot(sims[i].t,sims[i].I_ac,label='I_ac beta: '+str(beta[i]),linestyle=linestyle[i])
    #plt.plot(simulation_control.t,simulation_control.I,label='I '+I)
#plt.plot(simulation_control.t,simulation_control.I_ac,label='I_ac')
#plt.plot(simulation_control.t,simulation_control.R,label='R')
#plt.plot(simulation_control.t,simulation_control.B,label='B')
plt.legend(loc=0)
plt.show()

In [57]:
simulation_control.S[-1]

IndexError: invalid index to scalar variable.

In [ ]:
simulation_control.S[0] - simulation_control.B[-1] - simulation_control.R[-1]

In [ ]:
beta

In [ ]:
beta*rem_mob

In [ ]:
#%matplotlib tk
plt.title('Susceptibles')
for i in range(len(simulation)):
    plt.plot(simulation[i].dates,simulation[i].S,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    #plt.plot(simulation[i].dates,simulation[i].chi,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
#plt.scatter(state.Ir_dates,state.Ir)
for i in range(len(simulation_uk)):
    plt.plot(simulation_uk[i].dates,simulation_uk[i].S,linestyle='dashed',label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])

    
plt.axvline(x=newyear,color='blue',label='año nuevo')
plt.gca().xaxis.set_major_formatter( mdates.DateFormatter('%Y-%m-%d'))
plt.legend(loc=0)
plt.show()

In [ ]:
#%matplotlib tk
plt.title('Infectados Activos')
for i in range(len(simulation)):
    plt.plot(simulation[i].dates,simulation[i].I,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    #plt.plot(simulation[i].dates,simulation[i].chi,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
#plt.scatter(state.Ir_dates,state.Ir)
for i in range(len(simulation_uk)):
    plt.plot(simulation_uk[i].dates,simulation_uk[i].I,linestyle='dashed',label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])

plt.axvline(x=newyear,color='blue',label='año nuevo')
plt.gca().xaxis.set_major_formatter( mdates.DateFormatter('%Y-%m-%d'))
plt.legend(loc=0)
plt.show()

In [ ]:
#%matplotlib tk
plt.title('SeroPrevalence')
for i in range(len(simulation)):
    plt.plot(simulation[i].dates,simulation[i].I_ac/simulation[i].population,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    #plt.plot(simulation[i].dates,simulation[i].chi,label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
#plt.scatter(state.Ir_dates,state.Ir)
for i in range(len(simulation_uk)):
    plt.plot(simulation_uk[i].dates,simulation_uk[i].I_ac/simulation_uk[i].population,linestyle='dashed',label='RF: '+str(renewalFactor[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])

plt.gca().xaxis.set_major_formatter( mdates.DateFormatter('%Y-%m-%d'))
plt.legend(loc=0)
plt.show()

In [ ]:
state.Ir[0]

In [ ]:
type(simulation[0].I)

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


In [ ]:
SeroPrevalenceDet

In [ ]:
plt.plot(simulation)